In [4]:
# FatSecret API
from fatsecret import Fatsecret

In [223]:
!pip install hide_code
!jupyter nbextension install --py hide_code
!jupyter nbextension enable --py hide_code
!jupyter serverextension enable --py hide_code

     |████████████████████████████████| 62 kB 1.6 MB/s eta 0:00:01


Using legacy 'setup.py install' for hide-code, since package 'wheel' is not installed.
    Running setup.py install for hide-code ... done
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3 -m pip install --upgrade pip' command.
Installing /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/hide_code/ -> hide_code
Making directory: /usr/local/share/jupyter/nbextensions/hide_code/
Copying: /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/hide_code/hide_code_preprocessor.py -> /usr/local/share/jupyter/nbextensions/hide_code/hide_code_preprocessor.py
Copying: /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/hide_code/hide_code.py -> /usr/local/share/jupyter/nbextensions/hide_code/hide_code.py
Copying: /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/hide_code/hide_code_latex_exporter.py -> /usr/local/share/jupyter/nbextensions/h

Enabling notebook extension hide_code/hide_code...
      - Validating: OK
Enabling: hide_code
- Writing config: /Users/shiqiangxia/.jupyter
    - Validating...
      hide_code  OK


### Access Fastsecret Database API

In [226]:
consumer_key = 'c9cf8fed3da74f7085018f0d0e89f160'
consumer_secret = '1c0e39b64cf4456690692d8b3a7312b7'
print('key and secret for the API\nSenstive info, codes are hidden. ')

key and secret for the API
Senstive info, codes are hidden. 


In [225]:
try:
    fs = Fatsecret(consumer_key, consumer_secret)
except:
    print("Failed")
else:
    print("Success!")

Success!


### functions:

#### top level: 
Give names and amount, return nutrition facts
```
meal__nutrition_calculator(response, name_list, amount_list)
```

#### Middle level:
Calculate nutriton based on food and amount
```
meal_nutrition(food_list, amount_list, print_flag = True, break_down = False)
```

#### Bottom level
Find one certain food (or the closest n results)
```
find_food_nutrition(response, name, max_results = 1)
```







### Define MyFood Class

In [209]:
class MyFood:
    
    def __init__(self, name, food_id,serving_amount, cal, carb, fat, protein):
        self.name = name
        self.food_id = food_id
        self.serving_amount = serving_amount
        self.cal = cal
        self.carb = carb
        self.fat = fat
        self.protein = protein
        self.normalized_flag = False
        
    def show_nutrition_facts(self):
        print('---'*10)
        print('Food name: %s' % self.name)
        print('Food id: %s' % self.food_id)
        print('1 serving: %.2f g'  % self.serving_amount)
        print('Calorie: %.2f cal, Carbs: %.2f g, Fat: %.2f g, Protein: %.2f g' %(self.cal, self.carb, self.fat, self.protein))

    def normalize_serving(self):
        if self.serving_amount:
            goal = 100.0;
            current = self.serving_amount;
            if current!= goal:
                self.serving_amount = goal;
                ratio = goal/current ;
                self.cal *= ratio;
                self.carb *= ratio;
                self.fat  *= ratio;
                self.protein *= ratio;
            self.normalized_flag = True;
           
        else:
            print('This food "%s" has not serving amount info, we can not normalize it.' % self.name)
            
            
    def calculate_nutrition(self, amount_g):
        if self.serving_amount:
            ratio = amount_g/self.serving_amount
        else:
            print('Warning: This food has not serving amount info')
            ratio = 1.0; 
        return [self.cal*ratio, self.carb*ratio, self.fat * ratio, self.protein * ratio]    
        
                
                
        
        

        

### Obtain food data from FatSecret API

In [180]:
def find_food_nutrition(response, name, max_results = 1):
    try:
        foods = response.foods_search(name, max_results = max_results)
    except:
        print("Food %s is not found!"%name)
    else:
        id_list = [f['food_id'] for f in foods]
        if len(id_list)>=max_results:
            id_list = id_list[0:max_results]
    
    food_list = [];
    
    for idx in id_list:
        food = response.food_get(idx)
        all_servings = food['servings']['serving']
        if isinstance(all_servings,list):
            chosen_serving = all_servings[0]
        else:
            chosen_serving = all_servings
        try : 
            serving_amount = chosen_serving['metric_serving_amount']
            serving_amount = float(serving_amount)
        except:
            serving_amount = None
            
        name =  food['food_name']   
        cal = float(chosen_serving['calories'])
        carb = float(chosen_serving['carbohydrate'])
        fat = float(chosen_serving['fat'])
        protein = float(chosen_serving['protein'])
            
#         print('---'*10)
#         print('Food name: %s' % name)
#         print('Food id: %s' % idx)
#         print('1 serving: %s g'  % serving_amount)
#         print('Calorie: %s cal, Carbs: %sg, Fat: %sg, Protein: %sg' %(cal,carb,fat,protein))
        
        temp_food = MyFood(name, idx, serving_amount, cal, carb, fat, protein)
        
        food_list.append(temp_food)
    
    return food_list
        
    
        
    

### Calculate Meal Nutrition based on food list and amount list

In [211]:
def meal_nutrition(food_list, amount_list, print_flag = True, break_down = False):
    total = [0.0]*4
    if break_down:
        nutri_list = [];
        
    for ii, food in enumerate(food_list):
        nutrition = food.calculate_nutrition(amount_list[ii])
        for jj in range(4):
            total[jj] += nutrition[jj]
        if break_down:
            nutri_list.append(nutrition)
    
    print('Meal nutrition facts:')
    print('Calorie: %.2f, Carb: %.2f, Fat: %.2f, Protein: %.2f'%(total[0],total[1],total[2],total[3]))
    if break_down:
        print("\nNutrition breakdown:")
        for ii, food in enumerate(food_list):
            print('----'*10)
            temp_nutrition = nutri_list[ii]
            print('%s, amount %s g' %(food.name, amount_list[ii]))
            print('Calorie: %.2f (%.1f%%), Carb: %.2f (%.1f%%), Fat: %.2f (%.1f%%), Protein: %.2f (%.1f%%)'%
                  (temp_nutrition[0],100*temp_nutrition[0]/total[0],
                   temp_nutrition[1],100*temp_nutrition[1]/total[1],
                   temp_nutrition[2],100*temp_nutrition[2]/total[2],
                   temp_nutrition[3],100*temp_nutrition[3]/total[3]))
            
        

In [220]:
def meal__nutrition_calculator(response, name_list, amount_list):
    food_list = [];
    for foo in name_list:
        temp_food = find_food_nutrition(fs,foo)[0]
        food_list.append(temp_food)
        temp_food.normalize_serving()
        #temp_food.show_nutrition_facts()
    meal_nutrition(food_list,amount_list,break_down=True)

### Test codes

In [181]:
food_list = find_food_nutrition(fs,'pasta',max_results=3)

In [182]:
food_list[0].show_nutrition_facts()

------------------------------
Food name: Penne
Food id: 285243
1 serving: 140.00 g
Calorie: 220.00 cal, Carbs: 42.95 g, Fat: 1.29 g, Protein: 8.07 g


In [183]:
food_list[0].calculate_nutrition(100)

[157.14285714285714, 30.67857142857143, 0.9214285714285715, 5.764285714285714]

In [184]:
food_list[0].normalize_serving()

Normalizate : 1 serving amount = 100g


True

In [186]:
food_list[0].show_nutrition_facts()

------------------------------
Food name: Penne
Food id: 285243
1 serving: 100.00 g
Calorie: 157.14 cal, Carbs: 30.68 g, Fat: 0.92 g, Protein: 5.76 g


In [187]:
a= [1,2,3]
b = [1,2,3]
a+b

[1, 2, 3, 1, 2, 3]

In [203]:
meal_nutrition(food_list,[100,100,100],break_down=True)

Meal nutrition facts:
Calorie: 471.43, Carb: 92.04, Fat: 2.76, Protein: 17.29
Nutrition breakdown:
----------------------------------------
Penne, amount 100 g
Calorie: 157.14 (33.3%), Carb: 30.68 (33.3%), Fat: 0.92 (33.3%), Protein: 5.76 (33.3%)
----------------------------------------
Spaghetti, amount 100 g
Calorie: 157.14 (33.3%), Carb: 30.68 (33.3%), Fat: 0.92 (33.3%), Protein: 5.76 (33.3%)
----------------------------------------
Rotini, amount 100 g
Calorie: 157.14 (33.3%), Carb: 30.68 (33.3%), Fat: 0.92 (33.3%), Protein: 5.76 (33.3%)


In [221]:
food_cal(fs, ['rice dry','ground beef 93%','pasta'],[100,112,50])

Meal nutrition facts:
Calorie: 622.37, Carb: 95.77, Fat: 9.50, Protein: 34.99

Nutrition breakdown:
----------------------------------------
White Rice (Long-Grain, Dry, Unenriched, Parboiled), amount 100 g
Calorie: 373.80 (60.1%), Carb: 80.43 (84.0%), Fat: 1.04 (10.9%), Protein: 8.11 (23.2%)
----------------------------------------
Ground Beef 93%, amount 112 g
Calorie: 170.00 (27.3%), Carb: 0.00 (0.0%), Fat: 8.00 (84.2%), Protein: 24.00 (68.6%)
----------------------------------------
Penne, amount 50 g
Calorie: 78.57 (12.6%), Carb: 15.34 (16.0%), Fat: 0.46 (4.9%), Protein: 2.88 (8.2%)
